<a href="https://colab.research.google.com/github/DrAlexSanz/nlpv2-course/blob/master/Text_sumarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is extractive summarization, meaning that I score each sentence in the document and then I take the n best phrases as a summary. In this case I don't need a training/test set, the document that I need to summarize is my only data.

The alternative is generative summarization. The model reads the document and generates a summary that is not necessarily made of sentences in the document. Just generating the text is a whole new problem and that is when I would use seq2seq models.

In [35]:
import numpy as np
import pandas as pd
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
# Get the data

!wget https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

--2022-08-05 15:18:51--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv.1’

bbc_text_cls.csv.1  100%[===================>]   4.85M  5.40MB/s    in 0.9s    

2022-08-05 15:18:52 (5.40 MB/s) - ‘bbc_text_cls.csv.1’ saved [5085081/5085081]



In [38]:
df = pd.read_csv("bbc_text_cls.csv")

In [39]:
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [40]:
# My dataset is just one article
doc = df[df["labels"] == "business"]["text"].sample(random_state = 42)

In [41]:
def wrap_text(x):
    return textwrap.fill(x, replace_whitespace = False, fix_sentence_endings = True)

In [42]:
# Let's have a look at what I want to summarize. The title is included and it could just be a good summary or a baseline
print(wrap_text(doc.iloc[0]))

Christmas sales worst since 1981

UK retail sales fell in December,
failing to meet expectations and making it by some counts the worst
Christmas since 1981.

Retail sales dropped by 1% on the month in
December, after a 0.6% rise in November, the Office for National
Statistics (ONS) said.  The ONS revised the annual 2004 rate of growth
down from the 5.9% estimated in November to 3.2%. A number of
retailers have already reported poor figures for December.  Clothing
retailers and non-specialist stores were the worst hit with only
internet retailers showing any significant growth, according to the
ONS.

The last time retailers endured a tougher Christmas was 23 years
previously, when sales plunged 1.7%.

The ONS echoed an earlier
caution from Bank of England governor Mervyn King not to read too much
into the poor December figures.  Some analysts put a positive gloss on
the figures, pointing out that the non-seasonally-adjusted figures
showed a performance comparable with 2003. The Novembe

In [43]:
#Tokenize document into sentences

sents = nltk.sent_tokenize(doc.iloc[0].split("\n", 1)[1]) # Remove the title and split. This will produce a list [title, text] and I take the text

In [44]:
sents

['\nUK retail sales fell in December, failing to meet expectations and making it by some counts the worst Christmas since 1981.',
 'Retail sales dropped by 1% on the month in December, after a 0.6% rise in November, the Office for National Statistics (ONS) said.',
 'The ONS revised the annual 2004 rate of growth down from the 5.9% estimated in November to 3.2%.',
 'A number of retailers have already reported poor figures for December.',
 'Clothing retailers and non-specialist stores were the worst hit with only internet retailers showing any significant growth, according to the ONS.',
 'The last time retailers endured a tougher Christmas was 23 years previously, when sales plunged 1.7%.',
 'The ONS echoed an earlier caution from Bank of England governor Mervyn King not to read too much into the poor December figures.',
 'Some analysts put a positive gloss on the figures, pointing out that the non-seasonally-adjusted figures showed a performance comparable with 2003.',
 'The November-De

In [45]:
featurizer = TfidfVectorizer(stop_words = stopwords.words("english"), norm = "l1")

In [46]:
x = featurizer.fit_transform(sents)
x


<17x154 sparse matrix of type '<class 'numpy.float64'>'
	with 212 stored elements in Compressed Sparse Row format>

In [47]:
def get_sentence_score(tfidf_row):
    """given a row of the TfIdf matrix I calculate the mean (removing 0s). Remember that a row is a sentence"""
    row = tfidf_row[tfidf_row != 0]

    return row.mean()

In [48]:
# and now get the scores for each row. I can probably find a better way to do it with numpy but the where and the mask of np.mean() is confusing

scores = np.zeros(len(sents))

for i in range(len(sents)):
    row_score = get_sentence_score(x[i, :])
    scores[i] = row_score

In [54]:
# Sort the scores, and take the top n sentences.

# Alternatively:
# top N words or characters (for tags)
# top n% of sentences or words
# Use a threshold and take all the sentences above
# Use a threshold and multiply the averages by a weight factor

sort_idx = np.argsort(-scores) # It's sorting ASC by default

# Without sorting

ind = np.argpartition(scores, -5)[-5:]
ind

array([16, 12,  3,  2, 10])

In [50]:
# The result is not too bad! And I don't need to print the scores but why not?
# I don't need to sort by score, maybe I don't want to scramble the phrases

print("Here is the summary:")
for i in sort_idx[:5]:
    print(wrap_text("%.2f: %s" % (scores[i], sents[i])))

Here is the summary:
0.14: A number of retailers have already reported poor figures for
December.
0.13: However, reports from some High Street retailers highlight the
weakness of the sector.
0.12: The ONS revised the annual 2004 rate of growth down from the
5.9% estimated in November to 3.2%.
0.10: "Our view is the Bank of England will keep its powder dry and
wait to see the big picture."
0.10: And a British Retail Consortium survey found that Christmas 2004
was the worst for 10 years.


In [55]:
# Without sorting, in case it's a story or something like that
for i in ind:
    print(wrap_text("%.2f: %s" % (scores[i], sents[i])))


0.10: "Our view is the Bank of England will keep its powder dry and
wait to see the big picture."
0.10: And a British Retail Consortium survey found that Christmas 2004
was the worst for 10 years.
0.14: A number of retailers have already reported poor figures for
December.
0.12: The ONS revised the annual 2004 rate of growth down from the
5.9% estimated in November to 3.2%.
0.13: However, reports from some High Street retailers highlight the
weakness of the sector.
